LOAD MODEL

In [ ]:
from simple_model import CountryClassifier
import tensorflow as tf
from huggingface_hub import from_pretrained_keras

# model = CountryClassifier(124)
# model.load_weights(tf.train.latest_checkpoint("./checkpoints/country"))
model = from_pretrained_keras("rohanmyer/countryclassifier")

In [ ]:
# load labels
dataset = tf.keras.utils.image_dataset_from_directory(
        "./data/small_scrape/",
    )

labels = dataset.class_names

CHOOSE RANDOM IMG FROM DATASET

In [ ]:
from google.cloud import storage
from random import randint
from PIL import Image
import io

storage_client = storage.Client()
bucket = storage_client.get_bucket("geoguessr-imgs-2")
blobs = list(bucket.list_blobs(prefix="small_scrape/"))

In [ ]:
img_name = "small_scrape/Canada /63.74209595x-68.547180.jpg"

# img_blob = blobs[randint(0, len(blobs))].download_as_string() # select random img from dataset
img_blob = [b for b in blobs if b.name == img_name][0].download_as_string() # streetview with filepath img_name in dataset

stream = io.BytesIO(img_blob)
streetview = Image.open(stream).convert("RGB")
streetview

In [ ]:
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input

image = np.array(streetview)
image = tf.image.resize(image, [256, 256])
image = preprocess_input(tf.expand_dims(image, axis=0))

GRAD CAM

In [ ]:
# build model from inputs to output of last convolution layer
last_conv_layer = model.layers[0].get_layer("conv5_block3_out")
last_conv_layer_model = tf.keras.Model(model.layers[0].inputs, last_conv_layer.output)

In [ ]:
# build model from output of last conv layer to final predictions
classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
x = classifier_input
for layer_ind in [-3, -2, -1]:
    x = model.layers[0].layers[layer_ind](x)
classifier_model = tf.keras.Model(classifier_input, x)

In [ ]:
# get gradients of prediction wrt conv output
with tf.GradientTape() as tape:
    last_conv_layer_output = last_conv_layer_model(image)
    tape.watch(last_conv_layer_output)
    preds = classifier_model(last_conv_layer_output)
    top_pred_index = tf.argmax(preds[0])
    top_class_channel = preds[:, top_pred_index]
    
grads = tape.gradient(top_class_channel, last_conv_layer_output)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

In [ ]:
# multiply gradients by its feature map
last_conv_layer_output = last_conv_layer_output.numpy()[0]
pooled_grads = pooled_grads.numpy()
for i in range(pooled_grads.shape[-1]):
    last_conv_layer_output[:, :, i] *= pooled_grads[i]
    
# Average over all the filters to get a single 2D array
gradcam = np.mean(last_conv_layer_output, axis=-1)
# Clip the values (equivalent to applying ReLU) and normalize values
gradcam = np.clip(gradcam, 0, np.max(gradcam)) / np.max(gradcam)

In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

scale = 256 / gradcam.shape[0]
plt.imshow(streetview.resize((256, 256)))
plt.imshow(zoom(gradcam, zoom=(scale, scale)), alpha=0.5)

In [ ]:
{labels[i]: preds[0][i] for i in np.argsort(preds[0])[-5:]}